# Rode wijn in Portugal

Voor de Data Science casusopdracht heeft onze groep de rode wijn data set als onderzoekopdracht. In het eerste gedeelte van dit document zal er een een verkenning van de data plaatsvinden, waardoor er gerichte en interessante onderzoeksvragen opgesteld kunnen worden.

## Verkenning

Als eerste stap van de verkenning zal er onderzoek gedaan worden naar de huidige velden en de mogelijke verbeteringen aan de data set.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#voor vraag 3
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
#voor vraag 4
import holoviews as hv
from holoviews import dim, opts
from scipy import stats

hv.extension('bokeh')

In [ ]:
redwine = pd.read_csv('redwine.csv', sep=';')
redwine.head()

In [ ]:
redwine.set_index('id', inplace=True)

In [ ]:
# Eerst eens bekijken hoeveel records we op dit moment hebben
len(redwine)

In [ ]:
# Bekijken hoeveel records het scheelt met ontbrekende waardes
len(redwine.dropna())

Hier zit best een fors verschil tussen, waardoor er met meer detail naar de data gekeken moet worden.

Daarnaast valt het ons op dat de country en variety velden allemaal gelijke data bevatten, waardoor deze irrelevant zijn.

In [ ]:
redwine.drop(['country', 'variety'], axis=1, inplace=True)
redwine.head()

Hieronder tellen we het aantal rijen met missende data per kolom.

In [ ]:
redwine.isna().sum()

Het valt op dat alle missende data voorkomt bij designation en price, echter zijn er ook een aantal kolommen die niet het goede type zijn. Deze zullen zeer waarschijnlijk dus ook data bevatten die niet als float gecast kan worden.

In [ ]:
redwine.dtypes

In [ ]:
redwine[np.isnan(pd.to_numeric(redwine["density"], errors="coerce"))]

Hieruit blijkt dat er maar 1 rij is waar de density niet klopt, omdat het er maar 1 is, zullen we deze verwijderen uit de dataset.

In [ ]:
redwine.drop(1475, inplace=True)
redwine["density"] = redwine["density"].astype("float")

De andere velden met het type object bevatten meer invalide waardes:

In [ ]:
print(len(redwine[np.isnan(pd.to_numeric(redwine["citric acid"], errors="coerce"))]))
print(len(redwine[np.isnan(pd.to_numeric(redwine["alcohol"], errors="coerce"))]))

Omdat het bij alcohol ook om maar een aantal waardes gaat, is er gekozen om ook voor de records met ongeldige alcohol waardes de records te verwijderen.

In [ ]:
redwine = redwine[~np.isnan(pd.to_numeric(redwine["alcohol"], errors="coerce"))]
redwine["alcohol"] = redwine["alcohol"].astype("float")

### Tweede dataset

Als tweede dataset is er een set gepakt welke ook de zelfde chemische samenstellingen bevat als de huidige set, alleen is deze gericht op witte wijnen. 

In [ ]:
whitewine = pd.read_csv('winequality-white.csv', sep=';')
whitewine.head()

In tegenstelling tot bij de rode wijn, is er niet veel informatie over de regio en naamgeving van de wijn beschikbaar. Daarnaast is het een kwaliteitscijfer en niet een waardering van een reviewer, waardoor de cijfers niet direct te vergelijken zijn.

Bij de witte wijn set worden alle kolommen wel correct ingelezen:

In [ ]:
whitewine.dtypes


#### Histogrammen

Om meer inzicht te krijgen in de verdelingen van de stoffen, is voor elke stof een histogram opegsteld. Ook voor de pH, quality en alcohol waardes zijn grafieken gemaakt.

In [ ]:
sns.set()
for column in whitewine.columns.values:
    plt.title(column)
    if column in redwine:
        if redwine[column].dtype != "object":
            plt.hist(redwine[column], color="#ff335577")
    plt.hist(whitewine[column], color="#ffff3377")
    plt.show()

Het valt hierbij op dat de quality en citric acid geen rode wijnen heeft, omdat de quality niet aanwezig is als veld in de dataset, en omdat citric acid van het type object is. In tegenstelling tot alcohol en density is hier nog geen oplossing voor, omdat er veel meer datapunten missen.

Daarnaast is het interessant om te zien dat veel van deze grafieken geen goeie normaalverdeling vormen, wat doet vermoeden dat er een directe aanleiding is voor de verdeling van de stoffen. De grafiek van de pH waardes lijkt nog het meest normaal verdeeld.

## Onderzoeksvragen

Een eis voor het voltooien van deze casus is het opstellen (en beantwoorden) van onderzoeksvragen. Hierbij moet er rekening gehouden worden met de data die is aangeleverd en moet er een combinatie gemaakt worden met een externe set van data.

In het vorige gedeelte is een externe set verkend, welke betrekking heeft op witte wijnen in plaats van rode.

Met deze twee datasets en alle gegevens erin zijn we tot de volgende onderzoeksvragen gekomen:

- Welke proefpersonen geven gemiddeld meer of minder punten voor een wijn en hoeveel is dit?
- Zijn er groepen te onderscheiden in de wijn op basis van de stoffen en de wijnmakerijen?
- In hoeverre is de score van een Portugese Red te voorspellen op basis van de chemische kenmerken?
- Wat zijn de verschillen in verhoudingen van de chemische kenmerken bij rode t.o.v. witte wijnen?


## Onderzoeksvraag 1

Deze onderzoeksvraag luidt: "Welke proefpersonen geven gemiddeld meer of minder punten voor een wijn en hoeveel is dit?"

Door deze vraag te beantwoorden kan er gekeken worden of alle wijnen even eerlijk beoordeeld worden. Mocht er dan een vergelijking gaan komen tussen de verschillende wijnen waarbij het aantal punten een rol speelt kan er rekening mee gehouden worden.

Als eerste zullen we bekijken hoeveel reviewers er in totaal zijn en wat hun gemiddelde scores zijn:

In [ ]:
redwine['taster_name'].astype(str)
redwine_grouped = redwine.groupby(['taster_name'])[['taster_name', 'points']]
display(redwine_grouped.mean())

Hierboven zijn de gemiddelden per reviewer te zien.
Hierin valt op dat de gemiddelden allemaal binnen 5 punten van elkaar liggen.
Een verschil van een gemiddelde van 5 op een totaal van zo te zien 100 punten is niet genoeg om te zeggen dat er één of meerdere personen daadwerkelijk een andere waardering geven ten opzichte van de rest. Hieruit is voor deze deelvraag dus nog niks af te leiden.

Om er toch dieper op in te duiken zal er hieronder een histogram worden opgesteld voor elk proefpersoon. Hiermee kunnen we kijken of er ook echt uitschieters gegeven worden. Eventueel is er dan verder mee te onderzoeken of de lage(re) of hoge(re) scores zich allemaal bij bepaalde wijnen bevinden.

In [ ]:
plt.hist(redwine['points'], label='Uitgedeelde scores')
plt.legend()
plt.show()

Hierboven is te zien dat, zoals ook al af te leiden uit de gemiddelden hierboven, de meeste scores tussen de 85 en 90 zijn gegeven. 
Er ontstaat hier dan ook een mooie normaalverdeling in de uitgegedeelde punten.

In [ ]:
display(redwine_grouped.size())

Hierboven is per reviewer te zien hoeveel punten er daadwerkelijk uitgedeeld zijn. Hieruit is dus af te leiden of en persoon ook genoeg scores heeft uigedeeld heeft om betrouwbaar een gemiddelde vanaf te leiden.

Daarnaast is het interessant om te bekijken of de personen met een minder aantal uitgedeelde punten ook een hogere spreiding hebben in hun uitgedeelde punten.

In [ ]:
for taster_name, data in redwine_grouped:
    if data['points'].count() < 20:
        plt.hist(data['points'], label=taster_name)
        plt.legend()
        plt.show()

Hierboven is in de histogrammen te zien dat de meeste spreiding plaats vind bij de volgende personen:
- Carrie Dykes
- Fiona Adams
- Mike DeSimone

Ook valt het op dat alledrie de personen 5 of minder reviews hebben gegeven.

Wanneer we het dan verder gaan ontleden en dit afzetten ten opzichte van de wijnen die ze gewaardeerd hebben zien we het volgende voor 'Carrie Dykes' (beginnend bij de laagste score):

In [ ]:
display(redwine[redwine['taster_name'] == 'Carrie Dykes'].sort_values(['points']))

Nu is dus te zien dat de wijn genaamd 'Casal da Coelheira 2011 Red (Tejo)' de wijn is welke het minste gewaardeerd wordt door Carrie Dykes met 83 punten. Als er dan gekeken word naar alle scores gegeven aan deze wijn zien we de volgende data:

In [ ]:
display(redwine[redwine['title'] == 'Casal da Coelheira 2011 Red (Tejo)'].sort_values(['points']))

Deze wijn is geen goede peiler om te bekijken aangezien deze niet door een ander persoon gewaardeerd is. Om te voorkomen dat dit bij alles voorkomt word er bekeken of voor alle wijnen geld dat deze maar door een enkel persoon getest is.

In [ ]:
(redwine['title'] == 'Parras Vinhos 2010 Montaria Red (Alentejano)').sum()

Bij de eerstvolgende wijn beoordeeld door Carrie Dykes is al te zien dat er 2 personen deze wijn beoordeeld hebben. Met deze informatie is het dus al duidelijk dat er meerdere personen zijn die dezelfde wijn beoordelen.

Hieronder volgen de overige twee wijnen beoordeeld door Carrie:

In [ ]:
display(redwine[redwine['title'] == 'Parras Vinhos 2010 Montaria Red (Alentejano)'].sort_values(['points']))

Hier is te zien dat Carrie Dykes exact dezelfde score heeft gegeven op dezelfde wijn als 'Roger Voss' welke dan weer meer als 600 reviews op zijn naam heeft staan. Hier word er dus uitgegaan van een betrouwbare score.

Als volgende word de derde beoordeelde door Carrie Dykes behandeld.

In [ ]:
display(redwine[redwine['title'] == 'J, Portugal Ramos 2013 Ramos Reserva Red (Alentejano)'].sort_values(['points']))

Hier is helaas duidelijk dat er maar 1 persoon is geweest welke een score heeft gegeven aan deze wijn.
Voor nu is de conclusie dus dat de scores gegeven door deze persoon **onbetrouwbaar** zijn omdat op minder als de helft van de gegeven scores een referentie punt gevonden kan worden in andere reviewers.

---

Als volgende worden de wijnen van 'Fiona Adams' behandeld.

In [ ]:
display(redwine[redwine['taster_name'] == 'Fiona Adams'].sort_values(['points']))

Ook voor Fiona Adams zijn er weer 3 wijnen welke onderzocht zullen worden. Wat ook te zien is, is dat Fiona Adams in vergelijking met Carrie Dykes dus wel hogere punten uitdeeld.

Beginnende met de eerste wijn genaamd 'Quinta do Cavalinho':

In [ ]:
display(redwine[redwine['title'] == 'Quinta do Cavalinho 2013 Vale das Donas Red (Tejo)'].sort_values(['points']))

Helaas is ook hier weer Fiona Adams de enige persoon die een score heeft gegeven aan de wijn.

Door naar de volgende:

In [ ]:
display(redwine[redwine['title'] == 'Quinta do Casal Branco 2009 Quartilho Red (Ribatejo)'].sort_values(['points']))

Ook hier is het helaas weer alleen Fiona Adams die als enigste een score heeft uitgegeven.

De derde wijn als volgt:

In [ ]:
display(redwine[redwine['title'] == 'Barão de Vilar 2014 Cais da Ribeira Reserva Red (Douro)'].sort_values(['points']))

Bij de derde zien we dan wel dat Fiona Adams hetzelfde aantal punten heeft gegeven als 'Michael Schachner'.
Helaas komt het totaal er net als bij Carrie Dykes eropneer dat er **niet** voldoende data is om deze persoon als **betrouwbaar** aan te kunnen merken.

---

Dan als laatste word 'Mike DeSimone' behandeld:

In [ ]:
display(redwine[redwine['taster_name'] == 'Mike DeSimone'].sort_values(['points']))

Deze persoon heeft in totaal 5 wijnen beoordeeld.

beginnende met de wijn 'Sogevinus 2006 Curva Tinto Red (Douro)':

In [ ]:
display(redwine[redwine['title'] == 'Sogevinus 2006 Curva Tinto Red (Douro)'].sort_values(['points']))

Ondanks dat deze prima beoordeeld is op een schaal van 0 tot 100 is dit weer door 1 persoon gebeurd.

De volgende te onderzoeken is 'Companhia das Quintas 2011 Pancas Red (Lisboa)':

In [ ]:
display(redwine[redwine['title'] == 'Companhia das Quintas 2011 Pancas Red (Lisboa)'].sort_values(['points']))

Deze is wel door een andere beoordeeld. Dit is dezelfde 'Roger Voss' welke eerder ook al voorkwam bij 'Carrie Dykes' hierdoor kunnen we dus zeggen dat deze betrouwbaar is.

In [ ]:
display(redwine[redwine['title'] == 'Quinta de la Rosa 2011 Red (Douro)'].sort_values(['points']))

Deze wijn is weer door Mike DeSimone alleen beoordeeld.

In [ ]:
display(redwine[redwine['title'] == 'Real Companhia Velha 2013 Evel Reserva Red (Douro)'].sort_values(['points']))

Hierbij hetzelfde als de vorige, alleen door Mike DeSimone beoordeeld.

In [ ]:
display(redwine[redwine['title'] == 'Casa da Passarella 2012 Abanico Reserva Red (Dão)'].sort_values(['points']))

Hier hetzelfde.
Bij de persoon 'Mike DeSimone' is de conclusie dus ook **onbetrouwbaar** op basis van het te weinig kunnen peilen van de beoordelingen.

### Conlusie onderzoeksvraag 1
Als conclusie kunnen we dus zeggen dat de personen welke een minder "normale" normaalverdeling hadden in hun scores, door het te weinig geven van beoordelingen om deze goed te kunnen peilen, ook te onbetrouwbaar zijn om een duidelijk antwoord op te geven.

Wel is het zo dat de scores die vergeleken konden worden, gelijkwaardig zijn aan de scores gegeven door de meer ervaren personen.
Als conclusie valt hier dus uit op te maken, uitgaande dat de andere wijnen gelijkwaardig beoordeeld zouden worden, dat er geen personen zijn die een erg afwijkende score geven of een score welke eventueel bijgesteld zou kunnen worden.

## Onderzoeksvraag 2

Voor de tweede onderzoeksvraag zullen er groepen worden gemaakt op basis van de stoffen en de wijnmakerijen. Deze vraag is een clustering onderzoek, waardoor er bij voorbaad geen richtlijnen opgesteld kunnen worden voor een volledige beantwoording van de onderzoeksvraag. 

In [ ]:
from sklearn.cluster import KMeans
whitewine_clustering = whitewine

### Experimenten

Allereerst zullen er een aantal experimenten volgen rondom het clusteren van de witte wijn dataset op basis van verschillende eigenschappen.
Hierbij zullen er geen definitieve conclusies getrokken worden, maar ligt de nadruk op het vinden van interessante uitkomsten.
Wel is er soms extra toelichting over wat er te zien is in de verschillende plots.
De uitwerkingen in de clustering paragraaf zijn voorzien van correcte labels en een titel, de diagrammen hieronder dus niet.

In [ ]:
means = KMeans(n_clusters=3, random_state=0)
means.fit(whitewine_clustering[["alcohol", "sulphates"]])
means.cluster_centers_

In [ ]:
plt.scatter(whitewine_clustering["alcohol"], whitewine_clustering["sulphates"], c=means.labels_)

In dit cluster van drie groepen zijn er duidelijk 3 groepen gemaakt op basis van het alcohol percentage.

In [ ]:
means = KMeans(n_clusters=6, random_state=0)
means.fit(whitewine_clustering[["alcohol", "sulphates"]])
plt.scatter(whitewine_clustering["alcohol"], whitewine_clustering["sulphates"], c=means.labels_)

Wederom lijken de groepen enkel gebaseerd te zijn op het alcohol percentage.

In [ ]:
means = KMeans(n_clusters=4, random_state=0)
means.fit(whitewine_clustering[["alcohol", "quality"]])
plt.scatter(whitewine_clustering["alcohol"], whitewine_clustering["quality"], c=means.labels_)

In [ ]:
means = KMeans(n_clusters=4, random_state=0)
means.fit(whitewine_clustering[["pH", "alcohol"]])
plt.scatter(whitewine_clustering["pH"], whitewine_clustering["alcohol"], c=means.labels_)

In [ ]:
means = KMeans(n_clusters=4, random_state=0)
means.fit(whitewine_clustering[["alcohol", "residual sugar"]])
plt.scatter(whitewine_clustering["alcohol"], whitewine_clustering["residual sugar"], c=means.labels_)

In [ ]:
means = KMeans(n_clusters=4, random_state=0)
means.fit(whitewine_clustering[["pH", "residual sugar"]])
plt.scatter(whitewine_clustering["pH"], whitewine_clustering["residual sugar"], c=means.labels_)

In [ ]:
whitewine_clustering.groupby(means.labels_).describe()

Uit latere tests is gebleken dat er uitschieters in de data zitten, deze worden hieronder weggehaald uit de dataset:

In [ ]:
redwine_clustering = redwine[["points", "fixed acidity", "volatile acidity", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "pH", "sulphates", "alcohol"]]

whitewine_clustering = whitewine_clustering[whitewine_clustering["total sulfur dioxide"] < 200]
redwine_clustering = redwine_clustering[redwine_clustering["total sulfur dioxide"] < 250]

### Clustering

Hieronder zal het belangrijkste gedeelte van de clustering plaatsvinden, waarbij er gezocht zal worden naar een aantal groepen tussen de 3 en 5.
Voor elk van deze clustering uitkomsten zal worden onderzocht of er logische groepen ontstaan en of deze te herleiden zijn naar bepaalde eigenschappen. Dit zal zowel voor de rode als de witte wijn data set uitgevoerd worden.

In [ ]:
three_group_white_cluster = KMeans(n_clusters=3, random_state=0)
three_group_white_cluster.fit(whitewine_clustering)
whitewine_clustering.groupby(three_group_white_cluster.labels_).mean()

Hierbij valt op dat er voornamelijk verschillen tussen de groepen zitten voor de residual sugar, free sulfur dioxide, total sulfur dioxide en alchohol.

In [ ]:
four_group_white_cluster = KMeans(n_clusters=4, random_state=0)
four_group_white_cluster.fit(whitewine_clustering)
whitewine_clustering.groupby(four_group_white_cluster.labels_).mean()

Bij het clusteren in vier groepen valt op dat de grote verschillen wederom bij deze eigenschappen te vinden zijn.

In [ ]:
five_group_white_cluster = KMeans(n_clusters=5, random_state=0)
five_group_white_cluster.fit(whitewine_clustering)
whitewine_clustering.groupby(five_group_white_cluster.labels_).mean()

Bij vijf groepen is dit niet anders. Dat de grootste verschillen zitten in deze velden, blijkt voor het 2e, 3e en 4e genoemde element zelfs hetzelfde te zijn voor de rode wijn:

In [ ]:
three_group_red_cluster = KMeans(n_clusters=3, random_state=0)
three_group_red_cluster.fit(redwine_clustering)
redwine_clustering.groupby(three_group_red_cluster.labels_).mean()

In [ ]:
four_group_red_cluster = KMeans(n_clusters=4, random_state=0)
four_group_red_cluster.fit(redwine_clustering)
redwine_clustering.groupby(four_group_red_cluster.labels_).mean()

In [ ]:
five_group_red_cluster = KMeans(n_clusters=5, random_state=0)
five_group_red_cluster.fit(redwine_clustering)
redwine_clustering.groupby(five_group_red_cluster.labels_).mean()

Hieruit blijkt dat KMeans voor deze wijn datasets voornamelijk het onderscheid maakt op basis van de sulfur dioxide, en daarnaast gedeeltelijk op basis van alcohol en residual sugar. Om te visualiseren hoe groot de verschillen tussen de groepen zijn, zullen hieronder verschillende diagrammen volgen.

In [ ]:
plt.title("3 groups white wine clustering")
plt.xlabel("free sulfur dioxide")
plt.ylabel("total sulfur dioxide")
plt.scatter(whitewine_clustering["free sulfur dioxide"], whitewine_clustering["total sulfur dioxide"], c=three_group_white_cluster.labels_)

In [ ]:
plt.title("3 groups red wine clustering")
plt.xlabel("free sulfur dioxide")
plt.ylabel("total sulfur dioxide")
plt.scatter(redwine_clustering["free sulfur dioxide"], redwine_clustering["total sulfur dioxide"], c=three_group_red_cluster.labels_)

Het is hierbij leuk om te vermelden dat er redelijk vergelijkbare figuren en groepen voortkomen uit beide datasets. Hieronder zijn beide plots weergegeven met dezelfde assen:

In [ ]:
plt.title("3 groups white wine clustering with the same axes")
plt.ylim(0, 200)
plt.xlim(0, 100)
plt.xlabel("free sulfur dioxide")
plt.ylabel("total sulfur dioxide")
plt.scatter(whitewine_clustering["free sulfur dioxide"], whitewine_clustering["total sulfur dioxide"], c=three_group_white_cluster.labels_)

In [ ]:
plt.title("3 groups red wine clustering with the same axes")
plt.ylim(0, 200)
plt.xlim(0, 100)
plt.xlabel("free sulfur dioxide")
plt.ylabel("total sulfur dioxide")
plt.scatter(redwine_clustering["free sulfur dioxide"], redwine_clustering["total sulfur dioxide"], c=three_group_red_cluster.labels_)

Door deze grafieken met elkaar te vergelijken, is duidelijk te concluderen dat rode wijn over het algemeen veel minder sulfur dioxide bevat. Dit blijkt ook uit de gemiddelde waardes voor beide tabellen:

In [ ]:
redwine_clustering.mean()

In [ ]:
whitewine_clustering.mean()

In [ ]:
plt.title("5 groups white wine clustering")
plt.xlabel("free sulfur dioxide")
plt.ylabel("total sulfur dioxide")
plt.scatter(whitewine_clustering["free sulfur dioxide"], whitewine_clustering["total sulfur dioxide"], c=five_group_white_cluster.labels_)

In [ ]:
plt.title("5 groups red wine clustering")
plt.xlabel("free sulfur dioxide")
plt.ylabel("total sulfur dioxide")
plt.scatter(redwine_clustering["free sulfur dioxide"], redwine_clustering["total sulfur dioxide"], c=five_group_red_cluster.labels_)

Ook met 5 groepen valt duidelijk te zien dat de verdeling voornamelijk gebaseerd is op de totale hoeveelheid sulfur dioxide. Dit roept meteen de vraag op waarom er dan een zichtbaar verschil zat tussen de alcohol percentages. Om deze vraag te beantwoorden zal er gekeken worden naar de verhouding en relatie tussen sulfur dioxide en alcohol.

### Alcohol vs. sulfur dioxide

In dit laatste onderdeel van de tweede deelvraag zal er onderzoek gedaan worden naar de relatie van deze stoffen, omdat dit de stoffen waarom die de meeste invloed hadden op de groepsverdeling.

In [ ]:
plt.title("group 0 red wine alcohol histogram")
plt.xlabel("alcohol")
plt.ylabel("amount")
plt.hist(redwine_clustering[three_group_red_cluster.labels_ == 0]["alcohol"])

In [ ]:
plt.title("group 1 red wine alcohol histogram")
plt.xlabel("alcohol")
plt.ylabel("amount")
plt.hist(redwine_clustering[three_group_red_cluster.labels_ == 1]["alcohol"])

In [ ]:
plt.title("group 2 red wine alcohol histogram")
plt.xlabel("alcohol")
plt.ylabel("amount")
plt.hist(redwine_clustering[three_group_red_cluster.labels_ == 2]["alcohol"])

In de bovenstaande grafieken is duidelijk te zien dat er in de groep 0 een veel groter aandeel van sterkere rode wijnen is. Ditzelfde is ook te zien bij de witte wijnen, waar groep 0 aanzienlijk minder sterkere wijnen bevat:

In [ ]:
plt.title("group 0 white wine alcohol histogram")
plt.xlabel("alcohol")
plt.ylabel("amount")
plt.hist(whitewine_clustering[three_group_white_cluster.labels_ == 0]["alcohol"])

In [ ]:
plt.title("group 1 white wine alcohol histogram")
plt.xlabel("alcohol")
plt.ylabel("amount")
plt.hist(whitewine_clustering[three_group_white_cluster.labels_ == 1]["alcohol"])

In [ ]:
plt.title("group 2 white wine alcohol histogram")
plt.xlabel("alcohol")
plt.ylabel("amount")
plt.hist(whitewine_clustering[three_group_white_cluster.labels_ == 2]["alcohol"])

In [ ]:
whitewine_clustering.groupby(three_group_white_cluster.labels_)["alcohol"].mean()

In [ ]:
plt.title("white wine clustering")
plt.xlabel("total sulfur dioxide")
plt.ylabel("alcohol")
plt.scatter(whitewine_clustering["total sulfur dioxide"], whitewine_clustering["alcohol"], c=five_group_white_cluster.labels_)

In [ ]:
plt.title("red wine clustering")
plt.xlabel("total sulfur dioxide")
plt.ylabel("alcohol")
plt.scatter(redwine_clustering["total sulfur dioxide"], redwine_clustering["alcohol"], c=five_group_red_cluster.labels_)

De bovenstaande plots zijn interessant, want hierin is niet alleen nogmaals te zien dat witte wijn veel meer sulfur dioxide bevat, maar ook dat sterke rode wijnen vaak weinig sulfur dioxide bevatten, terwijl dit bij witte wijnen heel anders werkt.

In [ ]:
sns.jointplot(whitewine_clustering["total sulfur dioxide"], whitewine_clustering["alcohol"], kind="reg")

In [ ]:
sns.jointplot(redwine_clustering["total sulfur dioxide"], redwine_clustering["alcohol"], kind="reg")

Voor de meeste wijnen is wel te zeggen dat een hoge hoeveelheid alcohol in verband staat met een lage hoeveelheid sulfur dioxide.
Dit is niet het geval voor de hoeveelheid free sulfur dioxide, zoals hieronder te zien is:

In [ ]:
plt.title("white wine clustering")
plt.xlabel("free sulfur dioxide")
plt.ylabel("alcohol")
plt.scatter(whitewine_clustering["free sulfur dioxide"], whitewine_clustering["alcohol"], c=five_group_white_cluster.labels_)

In [ ]:
plt.title("red wine clustering")
plt.xlabel("free sulfur dioxide")
plt.ylabel("alcohol")
plt.scatter(redwine_clustering["free sulfur dioxide"], redwine_clustering["alcohol"], c=five_group_red_cluster.labels_)

Daarnaast valt het op de de groepen veel minder goed te onderscheiden zijn in deze plots, omdat de groepen dus voornamelijk gebaseerd zijn op de totale hoeveelheid sulfur dioxide.
Ook is er een verband tussen een hoge hoeveelheid alcohol en een lage hoeveelheid totaal sulfur dioxide.

### Koppeling met wijnmakerijen

In dit onderdeel van de onderzoeksvraag zal de terugkoppeling van de clusters naar de wijnmakerijen uitgevoerd worden.
Het is voor te stellen dat de clusters ook gedeeltelijk overeen komen met de wijnmakerijen, maar het kan ook zijn dat hier geen relatie in te ontdekken is. De wijnmakerij is alleen bekend bij de rode wijn dataset.

In [ ]:
redwine_original_with_no_outliers = redwine[redwine["total sulfur dioxide"] < 250]
groups = redwine_original_with_no_outliers.groupby(five_group_red_cluster.labels_)["winery"]
groups.describe()

In [ ]:
for group in range(0, 5):
    print(f"CLUSTER {group}")
    print(groups.value_counts()[group].head())

Hier zijn de wijnmakerijen matig te relateren aan de groepen, waarbij het opvalt dat "Wines & Winemakers" en "DFJ Vinhos" vaak verschuiven, ondanks dat zij veel wijnen in elke categorie hebben.

### Correlatie overzichten

In deze laatste paragraaf zijn nog correlatie overzichten te zien voor de stoffen.

In [ ]:
redwine_original_with_no_outliers.corr()

In [ ]:
whitewine_clustering.corr()

Hier valt te zien dat voor de rode wijnen de prijs en alcohol matig correleren met waardering van de wijn (points). Ook valt te concluderen dat er wel een verband is tussen total sulfur dioxide en alcohol, maar dat het niet om correlatie gaat (-0.264086).

### Conclusie

Als conclusie op het clusteren van de wijnen op basis van de stoffen en andere chemische eigenschappen zou er gezegd kunnen worden dat rode wijnen vaak minder sulfur dioxide bevatten.
Daarnaast is er een verband tussen enkele stoffen te ontdekken, waarop het kmeans algoritme de groepen indeeld.

Vervolgens is er nog onderzoek gedaan naar de relatie van de clusters met de wijnmakerijen. Hier zijn een aantal verbanden in te vinden, voornamelijk de verhouding van wijnen per cluster van "Wines & Winemakers" t.o.v. "DFJ Vinhos" is interessant.

Als laatste blijkt dat er correlatie tussen de prijs en alcohol met de waardering is.

Hiermee is onderzoeksvraag 2 beantwoord.

## Onderzoeksvraag 3

Deze onderzoeksvraag luidt "In hoeverre is de score van een Portugese Red te voorspellen op basis van de chemische kenmerken?"

Hierbij gaan we gebruik maken van supervised machine learning.
We hebben de dataframes nodig van alle chemische kenmerken met de prijs.


In [ ]:
redwine_priceprediction = redwine[['price','fixed acidity','volatile acidity','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']]
redwine_priceprediction

het valt gelijk op dat we aardig wat price waardes missen en zonder die kunnen we niks leren van onze dataset. dus zullen we voor deze vraag de records verwijderen waar een nulwaarde in voorkomt.

In [ ]:
print('voor het verwijderen van de nulwaardes zijn er : ' + str(len(redwine_priceprediction)) + ' records.')
redwine_priceprediction = redwine_priceprediction.dropna()
print('na het verwijderen van de nulwaardes zijn er : ' + str(len(redwine_priceprediction)) + ' records.')
redwine_priceprediction

nu gaan we kijken of we groepen kunnen maken op basis van de prijs hiervoor gaan we kijken wat voor prijzen er aanwezig zijn in onze dataset.

In [ ]:
redwine_priceprediction['price'].describe(include="all")

In [ ]:
plt.figure()
plt.title("prijs overzicht")
plt.xlabel("price")
plt.ylabel("amount")
plt.hist(redwine_priceprediction["price"])
ax = plt.gca()
ax.set_xlim(0, 150)

Het meeste zit dus tussen de 0 en 50 euro in, we kunnen de describe van hierboven gebruiken om groepen in te delen,<br />
groep 1 : 0-12 <br />
groep 2 : 12-17 <br />
groep 3 : 17-28 <br />
groep 4 : 28-40 <br />
groep 5 : 40+

In [ ]:
redwine_priceprediction.insert(1, 'prijsgroep', 'default value')

for i in redwine_priceprediction.index:
    if redwine_priceprediction.at[i, 'price'] < 12:
        redwine_priceprediction.at[i, 'prijsgroep'] = 1
    elif redwine_priceprediction.at[i, 'price'] < 17:
        redwine_priceprediction.at[i, 'prijsgroep'] = 2
    elif redwine_priceprediction.at[i, 'price'] < 28:
        redwine_priceprediction.at[i, 'prijsgroep'] = 3
    elif redwine_priceprediction.at[i, 'price'] < 40:
        redwine_priceprediction.at[i, 'prijsgroep'] = 4
    else:
        redwine_priceprediction.at[i, 'prijsgroep'] = 5


redwine_priceprediction.head()


In [ ]:
# maak een dictionary van prijsgroep naar prijsrange
lookup_redwine_priceprediction = {1: '0-12', 2: '12-17', 3: '17-28', 4: '28-40', 5: '40+'}
lookup_redwine_priceprediction


Nu hebben we onze dataset zo ingericht dat we er mee kunnen testen. Hiervoor gebruiken we K Nearest Neighbors.

### Voorspellen op alle chemische kenmerken.

We zullen nu gaan kijken als je alle chemische kenmerken pakt of K nearest Neighbors goed kan voorspellen hoe duur de wijn is.

In [ ]:
# maak de feature tabel (X) en de target tabel (Y)
X=redwine_priceprediction[['fixed acidity','volatile acidity','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']]
y=np.asarray(redwine_priceprediction['prijsgroep'], dtype="|S6")

# splits in train en test set
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 0)
# we gebruiken hier een random_state om te zorgen dat de waardes hetzelfde zijn elke keer en daar over kan worden uitgelegt.
print('Aantal trainwaarden {0:d}'.format(len(X_train)))
print('Aantal testwaarden {0:d}'.format(len(y_test)))

In [ ]:
# maak classifier object
knn = KNeighborsClassifier(n_neighbors = 9)
# train de classifier
knn.fit(X_train,y_train)
# laat de classifier de testset berekenen
y_knn = knn.predict(X_test)
# controleer de nauwkeurigheid met de test data
knn.score(X_test,y_test)

Een score van 0.30 is niet al te best. iets minder dan 30% wordt goed voorspelt als je alle chemische kenmerken meeneemt,
dit is wel beter dan gewoon gokken aangezien er 5 groepen zijn is de gokkans 20%.

In [ ]:
cm = confusion_matrix(y_test, y_knn)
print(cm)

Dit zijn de voorspellingen, je zou het liefst een diagonale lijn willen zien van linksboven naar rechtonder dan is de score 1. Het is zeker niet het geval, wat opvalt is dat de vierde prijsgroep (28-40) heel weinig wordt voorspelt.

In [ ]:
k = 9
while (k <= len(X_train)/2):
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train, y_train)
    y_knn = knn.predict(X_test)
    print("Score met k={}: {:.2f}".format(k, knn.score(X_test, y_test)))
    k += 50

knn = KNeighborsClassifier(n_neighbors = 9)
knn.fit(X_train, y_train)
y_knn = knn.predict(X_test)

Het aanpassen van de hoeveelheid neighbors maak niet veel uit voor de score van deze voorspelling.

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_gnb = gnb.predict(X_test)
print("Score met Gaussian NB is {:.2f} (was {:.2f} met KNN)".format(gnb.score(X_test, y_test), knn.score(X_test, y_test)))

De score wordt nog slechter met Gaussian NB

Overal dit is een zeer slechte score voor het voorspellen van de prijs dus zou ik niet voorspellen op basis van alle chemische kenmerken

### Voorspellen op 5 hoogst gecorreleerde chemische kenmerken

We gaan nu dezelfde data gebruiken maar op minder chemische kenmerken voorspellen om te kijken dat sommige chemische kenmerken meer invloed hebben als andere op de prijs. We zullen eerst naar de correlaties kijken


In [ ]:
redwine_priceprediction.corr()

In [ ]:
# maak de feature tabel (X) en de target tabel (Y)
X=redwine_priceprediction[['volatile acidity','total sulfur dioxide','density','sulphates','alcohol']]
y=np.asarray(redwine_priceprediction['prijsgroep'], dtype="|S6")

# splits in train en test set
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 0)
# we gebruiken hier een random_state om te zorgen dat de waardes hetzelfde zijn elke keer en daar over kan worden uitgelegt.
print('Aantal trainwaarden {0:d}'.format(len(X_train)))
print('Aantal testwaarden {0:d}'.format(len(y_test)))

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 9)
knn.fit(X_train, y_train)
y_knn = knn.predict(X_test)

cm = confusion_matrix(y_test, y_knn)
cm

Het valt weer op dat de vierde prijsgroep bijna geen voorspellingen krijgt.

In [ ]:
k = 9
while (k <= len(X_train)/2):
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train, y_train)
    y_knn = knn.predict(X_test)
    print("Score met k={}: {:.2f}".format(k, knn.score(X_test, y_test)))
    k += 50

Hier is de voorspelling score 32% iets hoger dan de bij het nemen van alle chemische kenmerken maar hoge prijzen worden nooit voorspelt. Waarschijnlijk zitten er minder wijnen in de hoge prijsgroepen en is de score daardoor hoger.

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_gnb = gnb.predict(X_test)
print("Score met Gaussian NB is {:.2f} (was {:.2f} met KNN)".format(gnb.score(X_test, y_test), knn.score(X_test, y_test)))

Het valt op dat de score ongeveer gelijk is met de voorspelling van alle chemische kenmerken. Maar als je alleen deze 5 stoffen neemt voorspelt die nooit een hoge prijs.

### Voorspellen met 2 chemische kenmerken

Als we alleen twee chemische stoffen die de hoogste correlatie met prijs nemen kan het zijn dat de voorspellingskans omhoog gaat. Dit zijn alchohol en volatile acidity.

In [ ]:
# maak de feature tabel (X) en de target tabel (Y)
X=redwine_priceprediction[['volatile acidity','alcohol']]
y=np.asarray(redwine_priceprediction['prijsgroep'], dtype="|S6")

# splits in train en test set
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 0)
# we gebruiken hier een random_state om te zorgen dat de waardes hetzelfde zijn elke keer en daar over kan worden uitgelegt.
print('Aantal trainwaarden {0:d}'.format(len(X_train)))
print('Aantal testwaarden {0:d}'.format(len(y_test)))

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 9)
knn.fit(X_train, y_train)
y_knn = knn.predict(X_test)

cm = confusion_matrix(y_test, y_knn)
cm

Dit is bijna hetzelfde als bij 5 stoffen

In [ ]:
k = 1
while (k <= len(X_train)/2):
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train, y_train)
    y_knn = knn.predict(X_test)
    print("Score met k={}: {:.2f}".format(k, knn.score(X_test, y_test)))
    k += 50

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_gnb = gnb.predict(X_test)
print("Score met Gaussian NB is {:.2f} (was {:.2f} met KNN)".format(gnb.score(X_test, y_test), knn.score(X_test, y_test)))

### Voorspellen op basis van alleen alcohol

De laatste groep waar we naar kijken is alleen alcohol.

In [ ]:
# maak de feature tabel (X) en de target tabel (Y)
X=redwine_priceprediction[['alcohol']]
y=np.asarray(redwine_priceprediction['prijsgroep'], dtype="|S6")

# splits in train en test set
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 0)
# we gebruiken hier een random_state om te zorgen dat de waardes hetzelfde zijn elke keer en daar over kan worden uitgelegt.
print('Aantal trainwaarden {0:d}'.format(len(X_train)))
print('Aantal testwaarden {0:d}'.format(len(y_test)))

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 9)
knn.fit(X_train, y_train)
y_knn = knn.predict(X_test)

cm = confusion_matrix(y_test, y_knn)
cm

Overal ongeveer hetzelfde, het is dus te concluderen dat alcohol de grootste invloed heeft op de prijs. 

In [ ]:
k = 1
while (k <= len(X_train)/2):
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train, y_train)
    y_knn = knn.predict(X_test)
    print("Score met k={}: {:.2f}".format(k, knn.score(X_test, y_test)))
    k += 50

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_gnb = gnb.predict(X_test)
print("Score met Gaussian NB is {:.2f} (was {:.2f} met KNN)".format(gnb.score(X_test, y_test), knn.score(X_test, y_test)))

Dit geeft een soortgelijke score als bij de 2 chemische kenmerken.

### Conclusie

Bij het voorspellen van de prijs krijg je het beste resultaat door naar alcohol gehalte te kijken, hierbij helpen de andere chemische stoffen heel weinig tot niet.

Je kan dus de prijs niet goed voorspellen aan de hand van de chemische kenmerken. Zelfs een ruwe schatting in een groepering van 5 prijsgroepen is de voorspelling maar 30% correct.

## Onderzoeksvraag 4

De laatste onderzoekvraag is "Wat zijn de verschillen in verhoudingen van de chemische kenmerken bij rode t.o.v. witte wijnen?"

We gaan beginnen met het kijken naar het gemiddelde van elke stof per wijnsoort.

### Gemiddelde

In [ ]:
redwine_stoffen = redwine[['fixed acidity','volatile acidity','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']]
display(redwine_stoffen.mean())

In [ ]:
whitewine_stoffen = whitewine[['fixed acidity','volatile acidity','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']]
display(whitewine_stoffen.mean())

Hieruit kan je gelijk al opmaken waar de grootste verschillen zitten en welke waardes minder interesant zijn om naar te kijken. 
Alcohol en pH waarden lijken aardig gelijk te liggen als je alleen naar de gemiddelde zou kijken.
Beide sulfur dioxides lijken meer uit elkaar te liggen en zal dan ook kenmerkender zijn voor een witte of rode wijn.

### Correlatie

We hebben al eerder de correlatie van alle data van de wijnen bekeken bij het onderzoeken van de stoffen zijn de correlaties ook belangerijk, bijvoordbeeld als er meer free sulfur dioxide in is, zal er dan ook meer total sulfur dioxides in de wijn zitten?

In [ ]:
redwine_stoffen.corr()

In [ ]:
whitewine_stoffen.corr()

Hierbij is de correlatie sterke hoever die van de 0 af ligt met als 1 en -1 een perfecte correlatie.
0.1 - 0.3 is lage correlatie
0.3 - 0.5 is matige correlatie
0.5 - 1 is sterke correlatie

Om de vraag van hiervoor te beantwoorde, ja als er meer free sulfur dioxide in de wijn zit is het meestal het geval dat er ook meer total sulfur dioxide in de wijn zit.

De sterkste correlatie die we hebben in witte wijnen is residual sugar met density 0.84 dus hoe meer residual sugar des te meer density in de rode wijnen is dit veel minder het geval. waar de correlatie maar een 0.28 is.

Dan is er nog een sterke relatie met density maar dan met alcohol negatief deze keer -0.78 dus hoe meer density des te lager de alcohol. dit is raar omdat dit helemaal niet het geval is bij rodewijn en is ook zeker interesant om naar te kijken.

Zonder dan naar de tabel te kijken kunnen we voorspellen dat witte wijnen met meer residual sugar ook minder alcohol bevatten, en dat klopt die heeft een correlatie van -0.45

### Verschillen per wijn

In de verkenning was hierop al ingegaan, daar was goed te zien dat er altijd overlap is tussen rode en witte wijnen is.
Bij density en residual sugar zien de histogrammer er niet zo mooi uit en daar gaan we eerst naar kijken.


In [ ]:
for column in whitewine_stoffen.columns.values:
    plt.title(column)
    plt.hist(redwine_stoffen[column], color="#ff335577")
    plt.hist(whitewine_stoffen[column], color="#ffff3377")
    plt.show()

### Combinatie van stoffen verschillen per wijn
Door 2 stoffen te pakken die iets anders bij rode en witte wijnen liggen kunnen we misschien iets interesants ontdekken dat meer inzicht geeft van chemische kenmerken per wijn.
Om wat inzicht te krijgen pakken we wat combinaties die interesant lijken. We hadden al gezien dat residual sugar en density heel verschillende correlaties hadden per wijn.

In [ ]:
redwinehv = hv.Points(redwine_stoffen, label='redwine', kdims=['residual sugar', 'density'])
whitewinehv = hv.Points(whitewine_stoffen, label='whitewine', kdims=['residual sugar', 'density'])
redwinehv.hist(num_bins=100, dimension=['residual sugar', 'density']) + whitewinehv.hist(num_bins=100, dimension=['residual sugar', 'density'])


je kan al gelijk zien dat er iets mis is met de waardes van deze datapunten. Deze zullen dus ook worden eruit gefilterd door alle records met density boven de 2 weg te halen en alle records boven de 40 van residual sugar weg te halen

In [ ]:

print("redwine lengte voor de removal: " + str(len(redwine_stoffen)))
print("whitewine lengte voor de removal: " + str(len(whitewine_stoffen)))
redwine_stoffen = redwine_stoffen[redwine_stoffen['density'] < 2]
whitewine_stoffen = whitewine_stoffen[whitewine_stoffen['density'] < 2]
redwine_stoffen = redwine_stoffen[redwine_stoffen['residual sugar'] < 40]
whitewine_stoffen = whitewine_stoffen[whitewine_stoffen['residual sugar'] < 40]
print("redwine lengte na de removal: " + str(len(redwine_stoffen)))
print("whitewine lengte na de removal: " + str(len(whitewine_stoffen)))

In [ ]:
redwinehv = hv.Points(redwine_stoffen, label='redwine', kdims=['residual sugar', 'density'])
whitewinehv = hv.Points(whitewine_stoffen, label='whitewine', kdims=['residual sugar', 'density'])
redwinehv.hist(num_bins=100, dimension=['residual sugar', 'density']) + whitewinehv.hist(num_bins=100, dimension=['residual sugar', 'density'])

Deze scatterplots zien er een stuk beter uit. De density bij rodewijnen ligt gemiddeld iets hoger en de residual sugar bij wittewijnen strekt veel hoger. witte wijn heeft een prachtige correlatie terwijl bij rode wijn die bijna niet terug is te vinden.

Om even terug te koppelen naar de correlatie van density en residual sugar, het kan zijn dat de outliers de correlatie hebben beinvloed en dat die daarom zo uit elkaar lagen voorheen, we zullen nog even snel die correlatie bekijken 

In [ ]:
redwine_stoffen.corr()

In [ ]:
whitewine_stoffen.corr()

rood : 0.284803 --> 0.261818
wit  : 0.838966 --> 0.833969

de outliers hebben weinig tot geen verschil gemaakt in de correlatie van density en residual sugar, dit zou ook niet moeten want de correlatie gaat over alle record en er zijn er maar een paar verwijderd. Je kan de correlatie ook goed terugzien in de scatterplots, bij whitewine hoe hoger de density hoe hoger de residual sugar. Maar bij redwine zit het allemaal een beetje op dezelfde residual sugar en is de correlatie dus laag.

In [ ]:
redwinehv = hv.Points(redwine_stoffen, label='redwine', kdims=['total sulfur dioxide','free sulfur dioxide'])
whitewinehv = hv.Points(whitewine_stoffen, label='whitewine', kdims=['total sulfur dioxide','free sulfur dioxide'])
redwinehv.hist(num_bins=100, dimension=['total sulfur dioxide','free sulfur dioxide']) + whitewinehv.hist(num_bins=100, dimension=['total sulfur dioxide','free sulfur dioxide'])



Deze hebben allebij een correlatie van 0.6 dus er is sprake van middelmatige correlatie ,hier valt op dat de verdeling hetzelfde is, maar de hoeveelheid hoger is in witte wijnen en dat er nog enkele uitschieters zijn bij rodewijn.

In [ ]:
redwinehv = hv.Points(redwine_stoffen, label='redwine', kdims=['total sulfur dioxide','fixed acidity'])
whitewinehv = hv.Points(whitewine_stoffen, label='whitewine', kdims=['total sulfur dioxide','fixed acidity'])
redwinehv.hist(num_bins=100, dimension=['total sulfur dioxide','fixed acidity']) + whitewinehv.hist(num_bins=100, dimension=['total sulfur dioxide','fixed acidity'])

hier valt op dat de fixed acidity een veel grotere spreiding heeft bij rode wijn dan bij wittewijn, en dat total sulfur dioxide hoger begint en eindigt bij witte wijn.

In [ ]:
redwinehv = hv.Points(redwine_stoffen, label='redwine', kdims=['volatile acidity','fixed acidity'])
whitewinehv = hv.Points(whitewine_stoffen, label='whitewine', kdims=['volatile acidity','fixed acidity'])
redwinehv.hist(num_bins=100, dimension=['volatile acidity','fixed acidity']) + whitewinehv.hist(num_bins=100, dimension=['volatile acidity','fixed acidity'])



de wolk is bij redwine veel gespreider, bij allebei de stoffen is dit het geval terwijl bij wittewijn een dense wolk wordt gevormt.

In [ ]:
redwinehv = hv.Points(redwine_stoffen, label='redwine', kdims=['alcohol','pH'])
whitewinehv = hv.Points(whitewine_stoffen, label='whitewine', kdims=['alcohol','pH'])
redwinehv.hist(num_bins=100, dimension=['alcohol','pH']) + whitewinehv.hist(num_bins=100, dimension=['alcohol','pH'])

deze twee stoffen zijn bijna precies gelijk aan elkaar, je zou kunnen zeggen dat wittewijnen een iets grotere wolk heeft maar dit kan goed komen omdat witte wijnen gewoon 2 keer zoveel records heeft en daarom iets groter lijkt.

In [ ]:
redwinehv = hv.Points(redwine_stoffen, label='redwine', kdims=['fixed acidity','sulphates'])
whitewinehv = hv.Points(whitewine_stoffen, label='whitewine', kdims=['fixed acidity','sulphates'])
redwinehv.hist(num_bins=100, dimension=['fixed acidity','sulphates']) + whitewinehv.hist(num_bins=100, dimension=['fixed acidity','sulphates'])


bij de rodewijn ligt de fixed acidity meer uit elkaar. de sulphates zien er meer of minder hetzelfde uit maar de rodewijn heeft iets meer uitschieters.

In [ ]:
redwinehv = hv.Points(redwine_stoffen, label='redwine', kdims=['residual sugar', 'alcohol'])
whitewinehv = hv.Points(whitewine_stoffen, label='whitewine', kdims=['residual sugar', 'alcohol'])
redwinehv.hist(num_bins=100, dimension=['residual sugar', 'alcohol']) + whitewinehv.hist(num_bins=100, dimension=['residual sugar', 'alcohol'])


niets heel interesants te vinden, alcohol is redelijk gelijk verdeelt terwijl residual sugar in de wittewijnen hoger is, wat al eerder geconstateerd was.


In [ ]:
redwinehv = hv.Points(redwine_stoffen, label='redwine', kdims=['density', 'alcohol'])
whitewinehv = hv.Points(whitewine_stoffen, label='whitewine', kdims=['density', 'alcohol'])
redwinehv.hist(num_bins=100, dimension=['density', 'alcohol']) + whitewinehv.hist(num_bins=100, dimension=['density', 'alcohol'])


Hier valt een sterke negatieve correlatie te ontdekken bij wittewijn is die iets sterker dan bij rode wijn, dus hoe hoger de density hoe lager de alcohol.

### Beschrijven van elke stof
Hieronder kan je de aantal recors, gemiddelde, standaarddeviatie, minimum en maximum zien van elke stof. Dit zal de eerdere vindingen ondersteunen en helpen een conclusie te trekken bij deze vraag.

In [ ]:
redwine_stoffen.describe(include="all")

In [ ]:
whitewine_stoffen.describe(include="all")

### Conclusie 
De conclusie is dat je niet altijd aan de stoffen kan zien of het een witte of rodewijn is, bij elke stof is er overlap zodanig dat het niet geheel kenmerkend is voor rood of witte wijn. Wel kan het zijn dat als een wijn bijvoorbeeld een hoge residual sugar heeft( bijv. 20+) dat het een witte wijn is.

Er zijn dus kenmerkende waardes van stoffen voor een specifieke wijn, maar niet een definitief antwoord voor elke mogelijke nieuwe wijn.

# Hypothese vraag

Ook is er vanuit de casus een hypothese mee gegeven welke behandeld zal worden. Deze is als volgt:

"In het afgelopen jaar was het bijzonder zonnig in Portugal. De verbouwers van wijn vragen zich af of dit de zuurgraad (pH) van heeft beïnvloed, met andere woorden: of de wijnen uit het afgelopen jaar een andere pH hebben dan. Ze doen daarom een steekproef en meten de zuurgraad in enkele rode wijnen van afgelopen jaar. Daar komen de volgende meetwaarden uit:

| Wijn | Suikergehalte |
|---|---|
| Casa Santa Vitória | 3.41 |
| Monte da Penha | 3.51 |
| Real Companhia Velha | 3.39 |
| Aveleda | 3.11 |
| Companhia das Quintas | 3.21 |
| J. Portugal Ramos | 3.50 |
| Sogrape | 3.46 |
| Casa Santos Lima | 3.37 |
| Quinta de Ventozelo | 3.71 |

Bepaal of deze wijnen een significant andere zuurgraad hebben dan gemiddeld. Kies als betrouwbaarheid 95%."

### Hypotheses

Uitgaande van bovenstaande informatie kunnen we dus twee hypothese vragen opstellen. Een 0 vraag en een A vraag, zie onderstaande:

H<sub>0</sub>: De wijnen hebben geen significant andere zuurgraad dan gemiddeld. ($\mu_{andere wijnen} == \mu_{nieuwe wijnen}$)

H<sub>a</sub>: De wijnen hebben wel een significant andere zuurgraad dan gemiddeld.  ($\mu_{andere wijnen} ≠ \mu_{nieuwe wijnen}$)

Om deze hypothese te kunnen bewijzen danwel ontkrachten zetten we onderstaande gegevens op een rijtje:
    
α = 5%  
𝜇 = ?  
Toetsing = Tweezijdig  
x̄ = ?  
se = ?  

Om deze te kunnen uitvoeren moeten we het gemiddelde van de oude wijnen weten om zo 𝜇 te kunnen bepalen.

In [ ]:
display('Waarde rode wijnen')
display(np.mean(redwine.pH))
display('Waarde nieuwe wijnen:')

newWines = pd.DataFrame([3.41, 3.51, 3.39, 3.11, 3.21, 3.50, 3.46, 3.37, 3.71])
display(np.mean(newWines))

Nu is het dus als volgt:  
α = 5% (ook wel 1.96 of 95%) 
𝜇 = 3.32  
Toetsing = Tweezijdig  
x̄ = 3.41  
se = ?  

Nu is het dus nog zaak om de standaardafwijking te bepalen:  

In [ ]:
redwine.std()

### Conclusie Hypotheses
Bovenstaande geeft aan dat de standaardafwijking 0.17 bedraagt op de pH waardes.
Dit betekent dat om binnen de aangegeven 95% te vallen de nieuwe wijnen gemiddeld buiten het bereik van 3.32 - (1.96*0.17 = 0.33) = 2.99 en 3.32+0.33 = 3.65.
Dus in totaal buiten het bereik van 2.99-3.65, aangezien de gemiddelde waarde op dit moment 3.41 is betekent dit dat H<sub>0</sub> geaccepteerd word en H<sub>a</sub> verworpen word.